# Dataset

In [21]:
# Carregando dataset mnist
import numpy as np
import matplotlib.pyplot as plt
import pyswarms as ps
import mnist_loader

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
training_data = list(training_data)

# Guardando dados de entrada e as saídas desejadas para cada amostra de treinamento
amostras = []
y_desejado = []
p = 0
for amostra in training_data:  
    aux = []
    for i in amostra[0]:
        aux.append(i[0])
        
    amostras.append(aux)
    
    aux = []
    for i in amostra[1]:
        aux.append(i[0])
    y_desejado.append(aux)
    
    if p == 499:
        break
    p += 1

amostras = np.array(amostras)
y_desejado = np.array(y_desejado)

# Parâmetros da rede neural

In [22]:
# Arquiteura da rede neural
n_entrada = 784                # número de entradas na camada de entrada
n_oculta = 30                  # número de neurônios na camada oculta
n_saida = 10                   # número de classes na camada de saídas

# Ativação, propagação e compração dos resultados

In [23]:
# Propagação direta da rede neural
def propagacao(particula):
    y_1 = ativacao1(particula)
    n = len(amostras)
    probs = ativacao2(y_1)
    erro = calculaErro(probs, n)
    return erro

def calculaErro(probs, n_amostras):
    # Matriz de confusão do erro
    correct_logprobs = np.zeros(n_amostras)
    aux = []
    for j in range(n_amostras):
        for k in range(len(probs[j])):
            if probs[j][k] == y_desejado[j][k]:
                aux.append(probs[j][k])
            else:
                aux.append((probs[j][k]-y_desejado[j][k])**2)
        correct_logprobs[j] = aux[j]
    
    # Calcula o erro da saída de rede
    #corect_logprobs = -np.log(probs[range(n_amostras), y_desejado]) # erro por saída
    erro = np.sum(correct_logprobs) / len(amostras)                   # erro médio total
    return erro

def ativacao2(y_1):
    # Ativação da camada de saída pela função de softmax
    exp_scores = np.exp(y_1)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return probs

def ativacao1(p):
    # Construindo vetores iniciais de pesos e bias a partir da lista de parâmetros
    W1 = p[0:23520].reshape((n_entrada,n_oculta))       # pesos oculta-entrada
    b1 = p[23520:23550].reshape((n_oculta,))            # bias oculta-entrada
    W2 = p[23550:23850].reshape((n_oculta,n_saida))     # pesos saída-oculta
    b2 = p[23850:23860].reshape((n_saida,))             # bias saída-oculta

    # Ativando camada oculta pela função de ativação sigmoid
    z1 = amostras.dot(W1) + b1      # nets da camada oculta
    a1 = np.tanh(z1)                # ativação da camada oculta
    y_1 = a1.dot(W2) + b2      # nets da camada de saída
    return y_1                 # saídas da rede neural

# Execução do PSO para otimização

In [24]:
"""import pso

otimizacao = pso.Pso(100, 100)
lb = np.array([-1] * dimensoes)
ub = np.array([1] * dimensoes)
posicao, custo = otimizacao.run(avaliacao, lb, ub)"""

def funcao_obj(enxame):
    n_particulas = enxame.shape[0]
    enxame = [propagacao(enxame[i]) for i in range(n_particulas)]
    return np.array(enxame)

# Definindo fator de aceleração 1 e 2 e peso de inércia para o PSO
parametrosPSO = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
dimensoes = (n_entrada * n_oculta) + (n_oculta * n_saida) + n_oculta + n_saida
n_particulas = 100
n_iteracoes = 50

# Iniciando PSO
otimizacao = ps.single.GlobalBestPSO(n_particles=n_particulas, dimensions=dimensoes, options=parametrosPSO)

# Realizando treinamento/otimização
custo, posicao = otimizacao.optimize(funcao_obj, iters=n_iteracoes, verbose=True)

2021-09-30 10:44:10,180 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|50/50, best_cost=0.09
2021-09-30 10:45:27,463 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.09000342198703525, best pos: [0.10654743 0.62092787 1.2747798  ... 1.38686022 0.72895938 1.34096168]


# Resultado do treinamento

In [25]:
print("Valor ótimo de minização: ", custo)
print("Pesos otimizados da camada oculta:\n", posicao[:23520])
print("Bias otimizados da camada oculta:\n", posicao[23520:23550])
print("Pesos otimizados da camada de saída:\n", posicao[23550:23850])
print("Bias otimizados da camada de saída:\n", posicao[23850:])

Valor ótimo de minização:  0.09000342198703525
Pesos otimizados da camada oculta:
 [0.10654743 0.62092787 1.2747798  ... 0.73729872 1.06783897 1.76580412]
Bias otimizados da camada oculta:
 [0.85476589 1.20788946 0.84565636 0.65468199 0.92206602 0.56331427
 1.26841844 1.53388275 0.93781721 1.62169157 1.25714957 1.27754267
 0.40466071 1.07002829 0.64533509 1.44556088 0.98863151 0.88100048
 1.25358453 0.80911349 0.95618924 0.62997516 1.18562282 0.49596633
 1.09236201 1.11387032 0.54616588 1.10226981 0.83829444 0.73735979]
Pesos otimizados da camada de saída:
 [ 1.17199042  0.5441611   0.96603615  1.39268983  1.51971124  1.52193096
  1.0824194   0.63691687  0.82911556  1.00713103  0.99636289  0.64288862
  1.69681279  0.44582437  0.93265744  0.96907176  0.98956137  1.07036405
  0.28795654  1.93738938  1.91944197  1.09085951  0.89130465  0.86535039
  1.25489785  1.32557819  0.87999412  1.1027951   0.1968381   1.19065259
  0.43864245  1.09295643  0.77114032  1.08199822  1.01341859  1.1179716

# Avaliação do treinamento e do teste

In [26]:
# Calculando a acurácia
def avaliacao(posicao):
    y_obtido = ativacao1(posicao)
    #y_obtido = ativacao2(y_obtido)
    resultado = np.argmax(y_obtido, axis=1)
    return y_obtido, resultado

In [27]:
# Acurácia do treino
n = len(amostras)
correct_log = np.zeros(len(y_desejado))

y_obtido, result_av = avaliacao(posicao)

print("Acurácia de treino: ", np.mean(y_obtido == y_desejado))

Acurácia de treino:  0.0


# Execução e avaliação do teste

In [28]:
# Guardando dados de entrada e as saídas desejadas para cada amostra de teste
test_data = list(test_data)
amostras = []
y_desejado = []
p = 0
for amostra in test_data:
    aux = []
    for i in amostra[0]:
        aux.append(i[0])
    amostras.append(aux)
    
    #converte decimal para binário e guarda em vetor de 10 posições
    y = np.zeros(10)
    vetor = list(map(int, bin(amostra[1]).replace("0b", "")))
    for i in range(1, len(vetor)+1):
        y[-i] = vetor[-i]
    y_desejado.append(y)
    
    if p == 199:
        break
    p += 1

amostras = np.array(amostras)
y_desejado = np.array(y_desejado)

# Acurácia do treino
correct_log = np.zeros(len(y_desejado))
y_obtido, result_av = avaliacao(posicao)

print("Acurácia de teste: ", np.mean(y_obtido == y_desejado))

Acurácia de teste:  0.0
